<a href="https://colab.research.google.com/github/phanimvsk/Capstone_Project/blob/Source-code/QA_model_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
nvidia-smi

Sat Nov 12 14:53:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash

%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-srfz642z/farm-haystack_a6c6aee3c27d4832875eec5bb73742cb
  Resolved https://github.com/deepset-ai/haystack.git to commit da6b0dc66f8789aa82f36554910a727a060a0bbd
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with sta

bash: line 2: fg: no job control
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-srfz642z/farm-haystack_a6c6aee3c27d4832875eec5bb73742cb
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.48.2 requires grpcio>=1.48.2, but you have grpcio 1.47.0 which is incompatible.


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(3)

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "/content/drive/MyDrive/input_data1"
#s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
#fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=None, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'content': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
# }
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:
print(docs)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/DL_aircAruvnKk.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/DL_k2P_pHQDlp0.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/DL_q6kJ71tEYqM.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/DL_oJNHXPs0XDk.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/NLP_dPaYDMk3vts&list=PLD392E2ACAEF0C689&index=13&ab_channel=nptelhrd.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/DL_VyWAvY2CF9c.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/Raw Data File_1.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/NLP_a9u96PnZL9g&list=PLD392E2ACAEF0C689&index=5&ab_channel=nptelhrd.txt
INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data1/NLP_6-E5fHyrmd4&list

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
!pip install urllib3==1.25.4
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Successfully uninstalled urllib3-1.26.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
docker 6.0.1 requires urllib3>=1.26.0, but you have urllib3 1.25.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-gdp6xznf
  Running command git clone --fi

In [ ]:
#from haystack.preprocessor.utils import convert_files_to_dicts
#from haystack.reader.farm import FARMReader
#from haystack.reader.transformers import TransformersReader
#from haystack.utils import print_answers

In [ ]:
from haystack.nodes import Seq2SeqGenerator
from transformers import pipeline, set_seed
#from haystack.reader.farm import FARMReader
from haystack.nodes import FARMReader
from haystack.utils import fetch_archive_from_http


set_seed(32)


#generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")
#reader = FARMReader(model_name_or_path="vblagoje/bart_lfqa", use_gpu=True)


reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "/content/drive/MyDrive/train_data"
# data_dir = "PATH/TO_YOUR/TRAIN_DATA"
reader.train(data_dir=data_dir, train_filename="Data_Annotated_3.json", use_gpu=True, n_epochs=1, save_dir="my_model4")
#

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'distilbert-base-uncased-distilled-squad' (DistilBert)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'distilbert-base-uncased-distilled-squad' (DistilBert model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: /content/drive/MyDrive/train_data/Data_Annotated_3.json 
Preprocessing dataset: 100%|██████████| 1/1 [00:36<00:00, 36.49s/ Dicts]
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING DEV DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No dev set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:ha

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Saving the model happens automatically at the end of training into the `save_dir` you specified
# However, you could also save a reader manually again via:
reader.save(directory="/content/drive/MyDrive/my_model4")


INFO:haystack.nodes.reader.farm:Saving reader model to /content/drive/MyDrive/my_model4


In [ ]:
# If you want to load it at a later point, just do:
new_reader = FARMReader(model_name_or_path="/content/drive/MyDrive/my_model4")


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: '/content/drive/MyDrive/my_model4' (DistilBert)
INFO:haystack.modeling.model.language_model:Loaded '/content/drive/MyDrive/my_model4' (DistilBert model) from local file system.
INFO:haystack.modeling.model.adaptive_model:Found files for loading 1 prediction heads
INFO:haystack.modeling.model.prediction_head:Loading prediction head from /content/drive/MyDrive/my_model4/prediction_head_0.bin
INFO:haystack.modeling.data_handler.processor:Initialized processor without tasks. Supply `metric` and `label_list` to the constructor for using the default task or add a custom task later via processor.add_task()
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
# Downloading script
!wget https://raw.githubusercontent.com/deepset-ai/haystack/main/haystack/utils/augment_squad.py

doc_dir = "data/tutorial2"

# Downloading smaller glove vector file (only for demonstration purposes)
glove_url = "https://nlp.stanford.edu/data/glove.6B.zip"
fetch_archive_from_http(url=glove_url, output_dir=doc_dir)

# Downloading very small dataset to make tutorial faster (please use a bigger dataset for real use cases)
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/squad_small.json.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Just replace the path with your dataset and adjust the output (also please remove glove path to use bigger glove vector file)
!python augment_squad.py --squad_path squad_small.json --output_path augmented_dataset.json --multiplication_factor 2 --glove_path glove.6B.300d.txt


--2022-10-31 17:49:04--  https://raw.githubusercontent.com/deepset-ai/haystack/main/haystack/utils/augment_squad.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12515 (12K) [text/plain]
Saving to: ‘augment_squad.py’

augment_squad.py    100%[===================>]  12.22K  --.-KB/s    in 0s      

2022-10-31 17:49:04 (70.6 MB/s) - ‘augment_squad.py’ saved [12515/12515]



INFO:haystack.utils.import_utils:Fetching from https://nlp.stanford.edu/data/glove.6B.zip to 'data/tutorial2'
INFO:haystack.utils.import_utils:Found data stored in 'data/tutorial2'. Delete this first if you really want to fetch new data.


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
tcmalloc: large alloc 1877803008 bytes == 0xfeb18000 @  0x7fb80d3b21e7 0x4b2590 0x5ad01c 0x5ad387 0x4d222f 0x51041f 0x4d14d6 0x57471e 0x50cb08 0x5b4ee6 0x4bad0a 0x4d3249 0x5

In [ ]:
# Loading a fine-tuned model as teacher e.g. "deepset/​bert-​base-​uncased-​squad2"
teacher = FARMReader(model_name_or_path="my_model", use_gpu=True)

# You can use any pre-trained language model as teacher that uses the same tokenizer as the teacher model.
# The number of the layers in the teacher model also needs to be a multiple of the number of the layers in the student.
student = FARMReader(model_name_or_path="huawei-noah/TinyBERT_General_6L_768D", use_gpu=True)

student.distil_intermediate_layers_from(teacher, data_dir=".", train_filename="augmented_dataset.json", use_gpu=True)
student.distil_prediction_layer_from(teacher, data_dir="data/squad20", train_filename="dev-v2.0.json", use_gpu=True)

student.save(directory="my_distilled_model")


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'my_model' (DistilBert)
INFO:haystack.modeling.model.language_model:Loaded 'my_model' (DistilBert model) from local file system.
INFO:haystack.modeling.model.adaptive_model:Found files for loading 1 prediction heads
INFO:haystack.modeling.model.prediction_head:Loading prediction head from my_model/prediction_head_0.bin
INFO:haystack.modeling.data_handler.processor:Initialized processor without tasks. Supply `metric` and `label_list` to the constructor for using the default task or add a custom task later via processor.add_task()
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'huawei-noah/TinyBERT_General_6L_768D' (Bert)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'huawei-noah/TinyBERT_General_6L_768D' (Bert model) from model hub.
Some weights of the model checkpoint at huawei-noah/TinyBERT_General_6L_768D were not used when initializing BertForQuestionAnswering: ['fit_denses.0.bias', 'cls.predictions.decoder.weight', 'fit_denses.6.weight', 'fit_denses.1.bias', 'fit_denses.2.bias', 'fit_denses.3.bias', 'fit_denses.3.weight', 'cls.predictions.transform.dense.weight', 'fit_denses.1.weight', 'fit_denses.4.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'fit_denses.5.weight', 'fit_denses.4.bias', 'fit_denses.0.weight', 'cls.predictions.transform.dense.bias', 'fit_denses.6.bias', 'fit_denses.5.bias', 'fit_denses.2.weight']
- This IS expected if you are initializing BertForQuestionAnsweri

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: augmented_dataset.json 
INFO:haystack.modeling.data_handler.processor:Couldn't find augmented_dataset.json locally. Trying to download ...
INFO:haystack.modeling.data_handler.processor:downloading and extracting file content to dir /
ERROR:haystack.modeling.data_handler.processor:Cannot download content. Unknown data source.


FileNotFoundError: ignored

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    #query="who won the first noble prize in physics?", params={"Retriever": {"top_k": 2}}
     query= "explain sigmoidal logistic thing?", params={"Retriever": {"top_k": 3}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.63 Batches/s]


In [ ]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Logistic Regression', 'type': 'extractive', 'score': 0.798193097114563, 'context': 'Logistic Regression ', 'offsets_in_document': [{'start': 0, 'end': 19}], 'offsets_in_context': [{'start': 0, 'end': 19}], 'document_id': 'ec5dadcbbdb5bfd9622f74fca749e96c', 'meta': {'name': 'Raw Data_3.txt'}}>,
             <Answer {'answer': 'To summarize this couple of modules logistic regression', 'type': 'extractive', 'score': 0.3127736449241638, 'context': 'To summarize this couple of modules logistic regression, so logistic regression is very ', 'offsets_in_document': [{'start': 0, 'end': 55}], 'offsets_in_context': [{'start': 0, 'end': 55}], 'document_id': '14904a950abfcc9c0c9bb7f0009102d', 'meta': {'name': 'Raw Data_3.txt'}}>,
             <Answer {'answer': 'I will explain', 'type': 'extractive', 'score': 0.24843671917915344, 'context': 'I will explain that in a second, what we mean by… ', 'offsets_in_document': [{'start': 0, 'end': 14}], 'offsets_in_context': [

In [ ]:
from haystack.utils import print_answers

# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(prediction, details="minimum")


Query: explain sigmoidal logistic thing?
Answers:
[   {'answer': 'Logistic Regression', 'context': 'Logistic Regression '},
    {   'answer': 'To summarize this couple of modules logistic regression',
        'context': 'To summarize this couple of modules logistic regression, '
                   'so logistic regression is very '},
    {   'answer': 'I will explain',
        'context': 'I will explain that in a second, what we mean by… '}]


In [ ]:
Query: what is civics?
Answers:
[   {   'answer': 'what you are estimating',
        'context': 'you can think it as y i hat. So, this is what you are '
                   'estimating and this is what is actual '},
    {   'answer': 'what the output the response variable stock value',
        'context': 'is what the stock should be this is what the output the '
                   'response variable stock value should '},
    {   'answer': 'what is offend',
        'context': 'And this is the point that is get captured in what is '
                   'offend, what is really described '}]